In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan  9 15:18:04 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A16          Off  | 00000000:35:00.0 Off |                    0 |
|  0%   43C    P8    13W /  62W |      8MiB / 15356MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A16          Off  | 00000000:36:00.0 Off |                    0 |
|  0%   

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir='/usr/lib/cuda'"

In [7]:
import torch
torch.cuda.is_available()

True

In [8]:
!pip install simpletransformers
import os
import matplotlib.pyplot as plt
import copy
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification,DataCollatorWithPadding
from datasets import load_dataset
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam,Nadam
from datasets import Dataset, DatasetDict
from tensorflow.keras.layers import Dense, BatchNormalization
from transformers import DataCollatorWithPadding

from sklearn.model_selection import train_test_split
from collections import Counter
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

from tensorflow.keras.layers import Dropout, Dense, Reshape, Bidirectional, GRU, GlobalMaxPooling1D,Flatten,Reshape,MultiHeadAttention,Layer,Activation, Multiply, Lambda, Permute


from tensorflow.keras import Input, Model, regularizers
#!pip install transformers==4.36.1

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


2024-01-09 15:18:09.385152: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-09 15:18:09.436438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-09 15:18:09.436490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-09 15:18:09.437975: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-09 15:18:09.448312: I tensorflow/core/platform/cpu_feature_guar

In [9]:
# !pip install nlpaug
# import nlpaug.augmenter.word as naw

In [10]:
# train = pd.read_csv("/content/rumoureval2019_train.csv")
# test = pd.read_csv("/content/rumoureval2019_test.csv")
# val = pd.read_csv("/content/rumoureval2019_val.csv")
# # print(train.shape)
# # train.sample(5)

# import pandas as pd
# from sklearn.model_selection import train_test_split

model_checkpoint = "roberta-base"  # Use RoBERTa model
batch_size = 8

In [11]:
# null_values = train.isnull().sum()
# print("Null values in each column:\n", null_values)
# # Remove rows with null entries
# train = train.dropna()


# null_values = val.isnull().sum()
# print("Null values in each column:\n", null_values)
# # Remove rows with null entries
# val = val.dropna()

# null_values = test.isnull().sum()
# print("Null values in each column:\n", null_values)
# # Remove rows with null entries
# test = val.dropna()

In [12]:
# print("Original labels:\n", train['label'].sample(5))

# # Create a mapping dictionary for label conversion
# label_mapping = {'support': 0, 'comment': 1, 'deny': 2, 'query': 3}

# # Replace string labels with numerical values
# train['label_number'] = train['label'].map(label_mapping)
# val['label_number'] = val['label'].map(label_mapping)
# test['label_number'] = test['label'].map(label_mapping)

# # Display the updated DataFrame with numerical labels
# print("\nUpdated labels:\n", train['label_number'].sample(5))
# train.to_csv("/content/rumoureval2019_train_modified.csv", index=False)
# val.to_csv("/content/rumoureval2019_val_modified.csv", index=False)
# test.to_csv("/content/rumoureval2019_test_modified.csv", index=False)

In [13]:
# val.head(10)

In [14]:
# Split the data into train, validation, and test sets
# train_data, test_data = train_test_split(train ,test_size=0.2, random_state=42)

# print(test_data.shape)
# test_data.tail(5)
# train_data, val_data  = train_test_split(train_data ,test_size=0.2, random_state=1, shuffle=False)
# train_data.to_csv('train_clean.csv',index=False)
# val_data.to_csv('val_clean.csv',index=False)
# test_data.to_csv('test_clean.csv',index=False)

dataset = load_dataset('csv', data_files={'train': './rumoureval2019_train_modified.csv','valid':'./rumoureval2019_val_modified.csv','test':'./rumoureval2019_test_modified.csv'})

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number'],
        num_rows: 4877
    })
    valid: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number'],
        num_rows: 1675
    })
})

In [15]:
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function_source(records):
    tokenized = tokenizer(records['source_text'], truncation=True,padding='max_length', return_token_type_ids=True, max_length=128)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'token_type_ids': tokenized['token_type_ids']
    }


pre_tokenizer_columns_source = set(dataset["train"].features)

pre_tokenizer_columns_source

encoded_dataset_source = dataset.map(preprocess_function_source, batched=True, )

print(encoded_dataset_source)

tokenizer_columns_source = list(set(encoded_dataset_source["train"].features) - pre_tokenizer_columns_source)



print("Columns added by tokenizer:", tokenizer_columns_source)

Map:   0%|          | 0/4877 [00:00<?, ? examples/s]

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 4877
    })
    valid: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1675
    })
})
Columns added by tokenizer: ['input_ids', 'attention_mask', 'token_type_ids']


In [16]:
def preprocess_function_reply(records):
    tokenized = tokenizer(records['reply_text'], truncation=True,padding='max_length', return_token_type_ids=True, max_length=128)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'token_type_ids': tokenized['token_type_ids']
    }


pre_tokenizer_columns_reply = set(dataset["train"].features)

pre_tokenizer_columns_reply

encoded_dataset_reply = dataset.map(preprocess_function_reply, batched=True, )

print(encoded_dataset_reply)

tokenizer_columns_reply = list(set(encoded_dataset_reply["train"].features) - pre_tokenizer_columns_reply)



print("Columns added by tokenizer:", tokenizer_columns_reply)

Map:   0%|          | 0/4877 [00:00<?, ? examples/s]

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 4877
    })
    valid: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['id', 'source_text', 'reply_text', 'label', 'label_number', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1675
    })
})
Columns added by tokenizer: ['input_ids', 'attention_mask', 'token_type_ids']


In [17]:
!pip install nlpaug

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")
from collections import Counter

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [18]:
# import pandas as pd

# # Assuming df is your DataFrame
# class_counts = train['label'].value_counts()

# print("Number of occurrences of each class:")
# print(class_counts)



batch_size = 32
max_length = 128

In [19]:
#source
input_ids_train_source = np.array(encoded_dataset_source['train']['input_ids'])
token_type_ids_train_source = np.array(encoded_dataset_source['train']['token_type_ids'])
attention_mask_train_source = np.array(encoded_dataset_source['train']['attention_mask'])


#reply
input_ids_train_reply = np.array(encoded_dataset_reply['train']['input_ids'])
token_type_ids_train_reply = np.array(encoded_dataset_reply['train']['token_type_ids'])
attention_mask_train_reply = np.array(encoded_dataset_reply['train']['attention_mask'])


In [20]:
#source
input_ids_valid_source = np.array(encoded_dataset_source['valid']['input_ids'])
token_type_ids_valid_source = np.array(encoded_dataset_source['valid']['token_type_ids'])
attention_mask_valid_source = np.array(encoded_dataset_source['valid']['attention_mask'])

#reply
input_ids_valid_reply = np.array(encoded_dataset_reply['valid']['input_ids'])
token_type_ids_valid_reply = np.array(encoded_dataset_reply['valid']['token_type_ids'])
attention_mask_valid_reply = np.array(encoded_dataset_reply['valid']['attention_mask'])

In [21]:
#source
input_ids_test_source = np.array(encoded_dataset_source['test']['input_ids'])
token_type_ids_test_source = np.array(encoded_dataset_source['test']['token_type_ids'])
attention_mask_test_source = np.array(encoded_dataset_source['test']['attention_mask'])

#reply
input_ids_test_reply = np.array(encoded_dataset_reply['test']['input_ids'])
token_type_ids_test_reply = np.array(encoded_dataset_reply['test']['token_type_ids'])
attention_mask_test_reply = np.array(encoded_dataset_reply['test']['attention_mask'])

In [22]:
model_checkpoint2 = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint2)
labels=  ['support', 'query', 'deny','comment']
tokenized_labels=tokenizer(labels,
          return_tensors='tf',
          return_token_type_ids=True,truncation=True,padding='max_length',  max_length=3)
tokenized_labels

2024-01-09 15:18:24.445987: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12744 MB memory:  -> device: 0, name: NVIDIA A16, pci bus id: 0000:37:00.0, compute capability: 8.6


{'input_ids': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[  101,  1619,   102],
       [  101, 15027,   102],
       [  101,  9762,   102],
       [  101,  7368,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=int32)>}

In [23]:
max_length = 128
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, Reshape, Bidirectional, GRU, GlobalMaxPooling1D,Flatten,Reshape
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Activation, Multiply, Lambda, Permute
from tensorflow.keras import regularizers

# Define the BERT model
#model_checkpoint = "bert-base-cased"
model_checkpoint2 = "bert-base-cased"

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint2)

# Get the BERT base model
bert_model2 = model.bert

from tensorflow.keras.layers import Layer

class BERTWrapperLayer(Layer):
    def __init__(self, bert_model, **kwargs):
        super(BERTWrapperLayer, self).__init__(**kwargs)
        self.bert_model = bert_model

    def call(self, inputs, **kwargs):
        input_ids, token_type_ids, attention_mask = inputs
        bert_output = self.bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        return bert_output.last_hidden_state
bert_last_hidden_state_labels = BERTWrapperLayer(bert_model2)([tokenized_labels['input_ids'],tokenized_labels['token_type_ids'], tokenized_labels["attention_mask"]])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
arr1 = bert_last_hidden_state_labels
arr2 = np.mean(arr1, axis=1)

In [25]:
arr2.shape

(4, 768)

In [26]:
# Pass your input through the BERT model wrapper
input_ids_source = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_source")
token_type_ids_source = Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids_source")
attention_mask_source = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_source")
input_ids_reply = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_reply")
token_type_ids_reply = Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids_reply")
attention_mask_reply = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_reply")
arr2_input = Input(shape=(4,768), dtype=tf.float32, name="arr2_input")
# Pass your input through the BERT model wrapper

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)
roberta_model=model.roberta

#deberta_pooler_output, deberta_last_hidden_state = DeBERTaWrapperLayer(deberta_model)([input_ids, attention_mask])
roberta_last_hidden_state_source = BERTWrapperLayer(roberta_model)([input_ids_source,token_type_ids_source, attention_mask_source])
roberta_last_hidden_state_reply = BERTWrapperLayer(roberta_model)([input_ids_reply,token_type_ids_reply, attention_mask_reply])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [27]:
roberta_last_hidden_state_source

<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_wrapper_layer_1')>

In [28]:
pooler_output_source = tf.reduce_mean(roberta_last_hidden_state_source, axis=1)
pooler_output_reply = tf.reduce_mean(roberta_last_hidden_state_reply, axis=1)

In [29]:
sliced_cls_source = roberta_last_hidden_state_source[:, 0]

# Slice the rest of the dimensions
cls_rest_of_dimension_source = roberta_last_hidden_state_source[:, 1:]

# Print or use the sliced outputs
print("First Dimension of source:", sliced_cls_source)

print("Rest of the Dimensions of source:", cls_rest_of_dimension_source)


sliced_cls_reply = roberta_last_hidden_state_reply[:, 0]

# Slice the rest of the dimensions
cls_rest_of_dimension_reply = roberta_last_hidden_state_reply[:, 1:]

# Print or use the sliced outputs
print("First Dimension of reply:", sliced_cls_reply)

print("Rest of the Dimensions of reply:", cls_rest_of_dimension_reply)

First Dimension of source: KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")
Rest of the Dimensions of source: KerasTensor(type_spec=TensorSpec(shape=(None, 127, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem_1/strided_slice:0', description="created by layer 'tf.__operators__.getitem_1'")
First Dimension of reply: KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem_2/strided_slice:0', description="created by layer 'tf.__operators__.getitem_2'")
Rest of the Dimensions of reply: KerasTensor(type_spec=TensorSpec(shape=(None, 127, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem_3/strided_slice:0', description="created by layer 'tf.__operators__.getitem_3'")


In [30]:
difference_cls = tf.abs(sliced_cls_source - sliced_cls_reply)
difference_cls

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf.math.abs')>

In [31]:
normalized_difference = tf.math.l2_normalize(difference_cls, axis=1)
dense_layer1 = tf.keras.layers.Dense(units=512, activation='Softmax')
# dense_layer2 = tf.keras.layers.Dense(units=256, activation='Softmax')

# # Apply the dense layer to the normalized difference
normalized_difference1 = dense_layer1(normalized_difference)
# output2 = dense_layer2(output1)
normalized_difference1
# output2

<KerasTensor: shape=(None, 512) dtype=float32 (created by layer 'dense')>

In [32]:
# layer = MultiHeadAttention(num_heads=2, key_dim=2)
# cross_attention_reply = layer(roberta_last_hidden_state_reply, roberta_last_hidden_state_reply)
# print(cross_attention_reply.shape)

In [33]:
# layer = MultiHeadAttention(num_heads=2, key_dim=2)
# cross_attention_source = layer(roberta_last_hidden_state_reply, roberta_last_hidden_state_source)
# print(cross_attention_source.shape)

In [34]:
# import tensorflow as tf

# class CrossAttentionLayer(tf.keras.layers.Layer):
#     def __init__(self, embed_size, num_heads):
#         super(CrossAttentionLayer, self).__init__()
#         self.num_heads = num_heads
#         self.embed_size = embed_size

#         # Query, Key, and Value weight matrices
#         self.WQ = tf.keras.layers.Dense(embed_size)
#         self.WK = tf.keras.layers.Dense(embed_size)
#         self.WV = tf.keras.layers.Dense(embed_size)

#         # Output weight matrix
#         self.WO = tf.keras.layers.Dense(embed_size)

#     def scaled_dot_product_attention(self, Q, K, V):
#         matmul_qk = tf.matmul(Q, K, transpose_b=True)
#         dk = tf.cast(tf.shape(K)[-1], tf.float32)
#         scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
#         attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
#         output = tf.matmul(attention_weights, V)
#         return output, attention_weights

#     def split_heads(self, x, batch_size):
#         x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
#         return tf.transpose(x, perm=[0, 2, 1, 3])

#     def call(self, inputs):
#         query, key, value = inputs

#         batch_size = tf.shape(query)[0]

#         # Linear projections
#         query = self.WQ(query)
#         key = self.WK(key)
#         value = self.WV(value)

#         # Split heads
#         query = self.split_heads(query, batch_size)
#         key = self.split_heads(key, batch_size)
#         value = self.split_heads(value, batch_size)

#         # Scaled Dot-Product Attention
#         scaled_attention, attention_weights = self.scaled_dot_product_attention(query, key, value)

#         # Merge heads
#         scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
#         concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embed_size))

#         # Linear projection
#         output = self.WO(concat_attention)

#         return output, attention_weights

# # Example usage
# seq_len = 128
# batch_size = 32
# embed_size = 256
# num_heads = 8

# cross_attention_layer = CrossAttentionLayer(embed_size, num_heads)

# # Dummy input tensors
# input_query = roberta_last_hidden_state_source
# input_key = roberta_last_hidden_state_reply
# input_value = roberta_last_hidden_state_source

# # Apply cross-attention
# cross_attention_source_output, attention_weights = cross_attention_layer([input_query, input_key, input_value])

# print("Input Query Shape:", input_query.shape)
# print("Output Shape:", cross_attention_source_output.shape)
# print("Attention Weights Shape:", attention_weights.shape)


In [35]:
import tensorflow as tf

class CrossAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, embed_size, num_heads):
        super(CrossAttentionLayer, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size

        # Query, Key, and Value weight matrices
        self.WQS = tf.keras.layers.Dense(embed_size)
        self.WKS = tf.keras.layers.Dense(embed_size)
        self.WVS = tf.keras.layers.Dense(embed_size)

        self.WQR = tf.keras.layers.Dense(embed_size)
        self.WKR = tf.keras.layers.Dense(embed_size)
        self.WVR = tf.keras.layers.Dense(embed_size)

        # Output weight matrix
        self.WOS = tf.keras.layers.Dense(embed_size)
        self.WOR = tf.keras.layers.Dense(embed_size)

    def scaled_dot_product_attention(self, Q, K, V):
        matmul_qk = tf.matmul(Q, K, transpose_b=True)
        dk = tf.cast(tf.shape(K)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, V)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        source, reply = inputs

        query_s = source
        query_r = reply

        value_s = source
        value_r = reply

        key_s = reply
        key_r = source

        batch_size = tf.shape(query_s)[0]

        # Linear projections
        query_s = self.WQS(query_s)
        key_s = self.WKS(key_s)
        value_s = self.WVS(value_s)

        query_r = self.WQR(query_r)
        key_r = self.WKR(key_r)
        value_r = self.WVR(value_r)

        # Split heads
        query_s = self.split_heads(query_s, batch_size)
        key_s = self.split_heads(key_s, batch_size)
        value_s = self.split_heads(value_s, batch_size)

        query_r = self.split_heads(query_r, batch_size)
        key_r = self.split_heads(key_r, batch_size)
        value_r = self.split_heads(value_r, batch_size)

        # Scaled Dot-Product Attention
        scaled_attention_s, attention_weights_s = self.scaled_dot_product_attention(query_s, key_s, value_s)
        #print(scaled_attention_s)
        scaled_attention_r, attention_weights_r = self.scaled_dot_product_attention(query_r, key_r, value_r)

        # Merge heads
        scaled_attention_s = tf.transpose(scaled_attention_s, perm=[0, 2, 1, 3])
        #print(scaled_attention_s)
        concat_attention_s = tf.reshape(scaled_attention_s, (batch_size, seq_len, self.embed_size))
        #print(concat_attention_s )

        scaled_attention_r = tf.transpose(scaled_attention_r, perm=[0, 2, 1, 3])
        concat_attention_r = tf.reshape(scaled_attention_r, (batch_size, seq_len, self.embed_size))

        # Linear projection
        output_s = self.WOS(concat_attention_s)
        output_r = self.WOR(concat_attention_r)

        return output_s, output_r

# Example usage
seq_len = 128
batch_size = 32
embed_size = 256
num_heads = 8

cross_attention_layer = CrossAttentionLayer(embed_size, num_heads)

# Dummy input tensors
input_query = roberta_last_hidden_state_reply
input_key = roberta_last_hidden_state_source
input_value = roberta_last_hidden_state_reply

# Apply cross-attention
cross_attention_source_output, cross_attention_reply_output = cross_attention_layer([roberta_last_hidden_state_source, roberta_last_hidden_state_reply])

print("Input Query Shape:", input_query.shape)
print("Output Shape:", cross_attention_reply_output.shape.as_list())
#print("Attention Weights Shape:", attention_weights.shape)


Input Query Shape: (None, 128, 768)
Output Shape: [None, 128, 256]


In [36]:
#word attention on source
word_embeddings = cross_attention_source_output

# Apply word-level attention
word_attention_scores = Dense(1, activation='relu')(word_embeddings)
word_attention_scores
word_attention_scores = Reshape((-1,))(word_attention_scores)
word_attention_weights = Activation('softmax')(word_attention_scores)
word_attention_weights = Reshape((-1, 1))(word_attention_weights)

# Tile the attention weights to match the dimensions of word_embeddings
word_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 256]))(word_attention_weights)

# Apply element-wise multiplication between word embeddings and attention weights
word_attention = Multiply()([word_embeddings, word_attention_weights])

# Sum up the word-level representations
word_attention_sum_source = Lambda(lambda x: tf.reduce_sum(x, axis=1))(word_attention)

In [37]:
#word attention on reply
word_embeddings = cross_attention_reply_output

# Apply word-level attention
word_attention_scores = Dense(1, activation='relu')(word_embeddings)
word_attention_scores
word_attention_scores = Reshape((-1,))(word_attention_scores)
word_attention_weights = Activation('softmax')(word_attention_scores)
word_attention_weights = Reshape((-1, 1))(word_attention_weights)

# Tile the attention weights to match the dimensions of word_embeddings
word_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 256]))(word_attention_weights)

# Apply element-wise multiplication between word embeddings and attention weights
word_attention = Multiply()([word_embeddings, word_attention_weights])

# Sum up the word-level representations
word_attention_sum_reply = Lambda(lambda x: tf.reduce_sum(x, axis=1))(word_attention)

In [38]:
word_attention_sum_reply

<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lambda_3')>

In [39]:
#concat word attentions with embeddings difference
combined_representation_cross_attention_embedding_diff = tf.concat([word_attention_sum_source, word_attention_sum_reply,normalized_difference1], axis=1)

In [40]:
combined_representation_cross_attention_embedding_diff

<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'tf.concat')>

In [41]:
# def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.9, dims=None):
#     # compute a basic fitness measure
#     if(weight_acc == None):
#         weight_acc = 0.99
#     weight_feat = 1 - weight_acc

#     if dims != None:
#         num_features = dims
#     else:
#         num_features = agent.shape[0]

#     acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
#     feat = (num_features - np.sum(agent))/num_features

#     fitness = weight_acc * acc + weight_feat * feat

#     return fitness, acc
# def AbSCA(num_agents, max_iter, train_data, train_label, obj_function=compute_fitness, trans_func_shape='s'):

#     short_name = 'SCA'
#     agent_name = 'Agent'
#     train_data, train_label = np.array(train_data), np.array(train_label)
#     num_features = train_data.shape[1]
#     trans_function = get_trans_function(trans_func_shape)

#     # setting up the objectives
#     weight_acc = None
#     if(obj_function == compute_fitness):
#         weight_acc = 0.99
#     obj = (obj_function, weight_acc)
#     # compute_accuracy is just compute_fitness with accuracy weight as 1
#     compute_accuracy = (compute_fitness, 1)

#     # initialize agents and Leader (the agent with the max fitness)
#     population = initialize(num_agents, num_features)
#     fitness = np.zeros(num_agents)
#     accuracy = np.zeros(num_agents)
#     Leader_agent = np.zeros((1, num_features))
#     Leader_fitness = float("-inf")
#     Leader_accuracy = float("-inf")

#     # initialize data class
#     data = Data()
#     data.train_X, data.val_X, data.train_Y, data.val_Y = train_test_split(
#         train_data, train_label, shuffle=False, test_size=0.2)

#     # create a solution object
#     solution = Solution()
#     solution.num_agents = num_agents
#     solution.max_iter = max_iter
#     solution.num_features = num_features
#     solution.obj_function = obj_function

#     # rank initial population
#     population, fitness, accs = sort_agents(population, obj, data)
#     Leader_agent = population[0].copy()
#     Leader_fitness = fitness[0].copy()

#     # start timer
#     start_time = time.time()

#     # Eq. (3.4)
#     a = 3

#     for iter_no in range(max_iter):
#         print('\n================================================================================')
#         print('                          Iteration - {}'.format(iter_no+1))
#         print('================================================================================\n')

#         # Eq. (3.4)
#         r1 = a-iter_no*((a)/max_iter)  # r1 decreases linearly from a to 0

#         # update the Position of search agents
#         for i in range(num_agents):
#             for j in range(num_features):

#                 # update r2, r3, and r4 for Eq. (3.3)
#                 r2 = (2 * np.pi) * np.random.random()
#                 r3 = 2 * np.random.random()
#                 r4 = np.random.random()

#                 # Eq. (3.3)
#                 if r4 < 0.5:
#                     # Eq. (3.1)
#                     population[i][j] = population[i][j] + (r1 * np.sin(r2) * abs(r3 * Leader_agent[j] - population[i][j]))
#                 else:
#                     # Eq. (3.2)
#                     population[i][j] = population[i][j] + (r1 * np.cos(r2) * abs(r3 * Leader_agent[j] - population[i][j]))

#                 temp = population[i][j].copy()
#                 temp = trans_function(temp)
#                 if temp > np.random.random():
#                     population[i][j] = 1
#                 else:
#                     population[i][j] = 0

#             # local search on every agent
#             print(f'\n******** Local Search on Agent {i+1} of Iteration {iter_no+1} ********\n')

#             agent = population[i].copy()
#             agentFit, agentAcc = compute_fitness(agent, data.train_X, data.val_X, data.train_Y, data.val_Y, weight_acc=0.99)

#             print(f'Initial fitness = {agentFit} | Initial accuracy = {agentAcc} | Nos of features = {int(np.sum(agent))}\n')

#             final_agent = adaptivebetaHC(agent, agentFit, agentAcc, data.train_X, data.val_X, data.train_Y, data.val_Y)

#             population[i] = final_agent.copy()

#         # update final information
#         population, fitness, accs = sort_agents(population, obj, data)
#         display(population, fitness, accs, agent_name)

#         if fitness[0] > Leader_fitness:
#             Leader_agent = population[0].copy()
#             Leader_fitness = fitness[0].copy()

#     # compute final accuracy
#     Leader_agent, _, Leader_accuracy = sort_agents(Leader_agent, compute_accuracy, data)
#     population, _, accuracy = sort_agents(population, compute_accuracy, data)

#     print('\n================================================================================')
#     print('                                    Final Result                                  ')
#     print('================================================================================\n')
#     print('Leader ' + agent_name +
#           ' Dimension : {}'.format(int(np.sum(Leader_agent))))
#     print('Leader ' + agent_name + ' Fitness : {}'.format(Leader_fitness))
#     print('Leader ' + agent_name +
#           ' Classification Accuracy : {}'.format(Leader_accuracy))
#     print('\n================================================================================\n')

#     # stop timer
#     end_time = time.time()
#     exec_time = end_time - start_time

#     # update attributes of solution
#     solution.best_agent = Leader_agent
#     solution.best_fitness = Leader_fitness
#     solution.best_accuracy = Leader_accuracy
#     solution.final_particles = population
#     solution.final_fitness = fitness
#     solution.final_accuracy = accuracy
#     solution.execution_time = exec_time

#     return solution

In [42]:
# soln_AbSCA = AbSCA(num_agents=1, max_iter=1, train_data=combined_representation_cross_attention_embedding_diff, train_label=train_labels)

In [43]:
# Add BiGRU layers for word-level attention
gru_units = 64
dropout_rate = 0.2

output = Reshape((-1, gru_units))(combined_representation_cross_attention_embedding_diff)
output = Bidirectional(GRU(gru_units, return_sequences=True))(output)
output = Dropout(dropout_rate)(output)
output = Bidirectional(GRU(gru_units, kernel_regularizer=regularizers.l2(0.01), return_sequences=True))(output)
output = Dropout(dropout_rate)(output)

# Apply sentence-level attention
sentence_attention_scores = Dense(1, activation='relu')(output)
sentence_attention_scores = Reshape((-1,))(sentence_attention_scores)
sentence_attention_weights = Activation('softmax')(sentence_attention_scores)
sentence_attention_weights = Reshape((-1, 1))(sentence_attention_weights)

# Tile the attention weights to match the dimensions of output
sentence_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 2 * gru_units]))(sentence_attention_weights)

# Apply element-wise multiplication between output and attention weights
sentence_attention = Multiply()([output, sentence_attention_weights])

# Sum up the sentence-level representations
sentence_attention_sum = Lambda(lambda x: tf.reduce_sum(x, axis=1))(sentence_attention)

# Apply global max pooling to capture the most important features
sentence_attention_pooling = GlobalMaxPooling1D()(sentence_attention)

combined_representation = tf.concat([sentence_attention_sum, sentence_attention_pooling], axis=1)
flat_labels=Flatten()(arr2_input)
dense_layer = tf.keras.layers.Dense(256, activation='relu')(flat_labels)
model1 = Model(inputs= [input_ids_source,token_type_ids_source,attention_mask_source,input_ids_reply,token_type_ids_reply,attention_mask_reply],outputs=[combined_representation])
dense_layer
dot_product=tf.keras.layers.Dot(axes=1)([model1.output, dense_layer])
#dot_product = tf.matmul(modelctR, dense_layer, transpose_b=True)

# Add dropout regularization after the hierarchical attention layer
#combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,dot_product], axis=1)
combined_representation = tf.concat([model1.output,dot_product], axis=1)

In [44]:
dot_product

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dot')>

In [45]:
model1.output

<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'tf.concat_1')>

In [46]:
output = Dropout(dropout_rate)(combined_representation)

# Add the final dense layer
output_units = 4
output1 = Dense(output_units, activation="softmax")(output)
output = BatchNormalization()(output)
# Create a new model with the modified architecture
#modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,arr2_input], outputs=output1)
modified_model = Model(inputs=[model1.input,arr2_input], outputs=output1)

# Define the optimizer
learning_rate = 2e-5
optimizer = Nadam(learning_rate=learning_rate)

# Define the loss function
loss = SparseCategoricalCrossentropy(from_logits=False)
#CategoricalCrossentropy
num_epochs = 10

In [47]:
# Compile the model
modified_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [48]:
replicated_array_train = np.tile(arr2, (len(encoded_dataset_reply["train"]), 1,1))
replicated_array_val = np.tile(arr2, (len(encoded_dataset_reply["valid"]), 1,1))
replicated_array_val.shape

(1440, 4, 768)

In [51]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a checkpoint callback
checkpoint_filepath = 'best_weights'
model_checkpoint = ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='val_accuracy', mode='max', save_format='tf', verbose=1)


# X and label_train remain the same as in your code
X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,replicated_array_train]
label_train = np.asarray(encoded_dataset_source['train']["label_number"]).astype(np.float32)
label_val = np.asarray(encoded_dataset_source['valid']["label_number"]).astype(np.float32)

# Train the modified model with the checkpoint callback
history = modified_model.fit(X, label_train,
                              validation_data=([input_ids_valid_source, token_type_ids_valid_source, attention_mask_valid_source,
                                                input_ids_valid_reply, token_type_ids_valid_reply, attention_mask_valid_reply,
                                                replicated_array_val], label_val),
                              epochs=num_epochs, batch_size=8, callbacks=[model_checkpoint])

# Load the best weights
modified_model.load_weights(checkpoint_filepath)

# Now you can use the modified_model for testing or further evaluation
# modified_model.evaluate(test_data, test_labels)


Epoch 1/10
610/610 [==============================] - ETA: 0s - loss: 2.5748 - accuracy: 0.7519
Epoch 1: val_accuracy improved from -inf to 0.81597, saving model to best_weights
INFO:tensorflow:Assets written to: best_weights/assets


INFO:tensorflow:Assets written to: best_weights/assets


610/610 [==============================] - 560s 919ms/step - loss: 2.5748 - accuracy: 0.7519 - val_loss: 2.2617 - val_accuracy: 0.8160
Epoch 2/10
610/610 [==============================] - ETA: 0s - loss: 2.1672 - accuracy: 0.7841
Epoch 2: val_accuracy did not improve from 0.81597
610/610 [==============================] - 527s 863ms/step - loss: 2.1672 - accuracy: 0.7841 - val_loss: 1.9587 - val_accuracy: 0.8139
Epoch 3/10
610/610 [==============================] - ETA: 0s - loss: 1.8001 - accuracy: 0.8245
Epoch 3: val_accuracy did not improve from 0.81597
610/610 [==============================] - 526s 862ms/step - loss: 1.8001 - accuracy: 0.8245 - val_loss: 1.8296 - val_accuracy: 0.7458
Epoch 4/10
610/610 [==============================] - ETA: 0s - loss: 1.4904 - accuracy: 0.8559
Epoch 4: val_accuracy did not improve from 0.81597
610/610 [==============================] - 526s 862ms/step - loss: 1.4904 - accuracy: 0.8559 - val_loss: 1.7119 - val_accuracy: 0.7181
Epoch 5/10
610/610 

2024-01-09 17:09:53.497983: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open best_weights: FAILED_PRECONDITION: best_weights; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [52]:
replicated_array_test = np.tile(arr2, (len(encoded_dataset_reply["test"]), 1,1))

In [53]:
# Evaluate the model on the test dataset
#X_test = [input_ids_test_source,token_type_ids_test_source, attention_mask_test_source,input_ids_test_reply,token_type_ids_test_reply, attention_mask_test_reply,v_p_s_test,v_neg_s_test,v_neu_s_test,v_p_r_test,v_neg_r_test,v_neu_r_test,clip_s_test,clip_r_test,replicated_array_test]
X_test = [input_ids_test_source,token_type_ids_test_source, attention_mask_test_source,input_ids_test_reply,token_type_ids_test_reply, attention_mask_test_reply,replicated_array_test]
label_test = np.asarray(encoded_dataset_source['test']["label_number"]).astype(np.float32)

test_results = modified_model.evaluate(X_test,label_test)

# Print the test accuracy
test_accuracy = test_results[1] * 100
print("Test Accuracy: {:.2f}%".format(test_accuracy))

53/53 [==============================] - 51s 874ms/step - loss: 2.2055 - accuracy: 0.8436
Test Accuracy: 84.36%


In [54]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
# y_true = tf_test_dataset.map(lambda x, y: y)
# y_true = np.concatenate(list(y_true.as_numpy_iterator()))

# Calculate the F1 score
f1score = f1_score(label_test, y_pred, average='weighted')
f1_score_macro =precision_recall_fscore_support(label_test, y_pred, average='macro')

print("F1 Score: {:.2f}".format(f1score))
print("F1 macro Score: {:.2f}",(f1_score_macro))

53/53 [==============================] - 52s 872ms/step
F1 Score: 0.81
F1 macro Score: {:.2f} (0.5597137263879143, 0.42025297804212397, 0.4388216327912988, None)


In [55]:
import numpy as np
from sklearn.metrics import f1_score

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
#y_true = tf_test_dataset.map(lambda x, y: y)
#y_true = np.concatenate(list(y_true.as_numpy_iterator()))
# Calculate the F1 score for each label
y_true = label_test
num_labels = 4 # Number of labels in the dataset
f1_scores = []
for label in range(num_labels):
    label_true = (y_true == label)
    label_pred = (y_pred == label)
    label_f1_score = f1_score(label_true, label_pred)
    f1_scores.append(label_f1_score)

# Print the F1 score for each label
for label, f1_score in enumerate(f1_scores):
    print("F1 Score for label {}: {:.2f}".format(label, f1_score))

53/53 [==============================] - 46s 873ms/step
F1 Score for label 0: 0.02
F1 Score for label 1: 0.91
F1 Score for label 2: 0.34
F1 Score for label 3: 0.48


In [ ]:
support 0.40 deny 0.55 comment 0.90 query 0.22 0.385 
support 0.02 commnet 0.34 deny 0.91 query 0.48  
'support': 0, 'comment': 1, 'deny': 2, 'query': 3}

In [ ]:
modified_model.save("lata", save_format="tf")


In [ ]:
# from tensorflow.keras.models import load_model



# # Load the model
# loaded_model = load_model("lata")

# # Now you can use the loaded_model for inference without retraining


In [56]:
predictions=modified_model.predict(X_test)

53/53 [==============================] - 46s 863ms/step


In [57]:
predictions

array([[0.0808573 , 0.8156917 , 0.07039624, 0.03305472],
       [0.16664861, 0.7586253 , 0.04977947, 0.02494655],
       [0.12760513, 0.55149883, 0.09476161, 0.22613445],
       ...,
       [0.07732699, 0.843201  , 0.05615843, 0.02331355],
       [0.06859208, 0.8379199 , 0.06497687, 0.02851118],
       [0.05840518, 0.86761695, 0.04689997, 0.02707791]], dtype=float32)

In [58]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Assuming y_true and y_pred are NumPy arrays
# If they are not NumPy arrays, you can convert them using np.array(your_list)

# Example true labels (ground truth) for a multiclass problem
y_true = np.array(label_test)

# Example predicted probabilities from your model
# This assumes you have a model predicting probabilities for each class
y_probs = np.array(predictions)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_probs, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


Accuracy: 0.8435820895522388
Precision: 0.7977051079568231
Recall: 0.8435820895522388
F1-Score: 0.8061053436100679


In [59]:
with open("metrics.txt", "w") as file:
    file.write(f'Accuracy: {accuracy}\n')
    file.write(f'Precision: {precision}\n')
    file.write(f'Recall: {recall}\n')
    file.write(f'F1-Score: {f1}\n')

print("Metrics saved to metrics.txt")

Metrics saved to metrics.txt
